## Import Statements

In [1]:
# Import Statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import chart_studio.plotly as py
import re
import geopy
from geopy.distance import geodesic

from IPython.display import display, HTML

### Read in other2142_runways.csv

`This .csv file contains the 599 remarks that reference runways (of the 2142 OTHER Standard Format Remarks).`

In [2]:
# Read in Files
rwy599 = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/other2142_runways.csv')
rwy599

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance
0,0,Aircraft observed a UAS 100 feet above the Air...,['RWY30'],NaN,NaN,NaN,['2 NM']
1,1,Aircraft observed an UAS at 200 feet above the...,['RWY8'],NaN,NaN,NaN,[]
2,2,Aircraft observed a UAS off his right side at ...,['RWY23'],NaN,NaN,NaN,[]
3,3,"Aircraft observed a red and white UAS at 1,000...","['RWY18L', 'RWY18C']",NaN,NaN,NaN,[]
4,4,"Aircraft observed a black UAS at 1,000 feet wh...",['RWY 32R'],PAM,PAM,RWY 32R,['3 NM']
...,...,...,...,...,...,...,...
594,596,Aircraft reported a fixed wing UAS pass within...,['RWY 10'],SAV,SAV,RWY 10,['1 NM']
595,597,Aircraft reported a grey quad-copter UAS off t...,['RWY 27L'],SFB,SFB,RWY 27L,[]
596,598,Aircraft reported a black quad-copter UAS off ...,['RWY 3R'],NaN,NaN,NaN,[]
597,599,Aircraft reported a NMAC with a UAS while on a...,['RWY05'],NaN,NaN,NaN,['6 NM']


#### We first explore the ***102*** records for which there is NO NULL (or blank) INFORMATION

`These records will have useable information in all 6 columns of interest in the rwy599 dataframe`

In [3]:
# First we need to replace all [] in UAS_Distance field with NaN
rwy599_df = rwy599.mask(rwy599.applymap(str).eq('[]'))
rwy599_df

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance
0,0,Aircraft observed a UAS 100 feet above the Air...,['RWY30'],NaN,NaN,NaN,['2 NM']
1,1,Aircraft observed an UAS at 200 feet above the...,['RWY8'],NaN,NaN,NaN,NaN
2,2,Aircraft observed a UAS off his right side at ...,['RWY23'],NaN,NaN,NaN,NaN
3,3,"Aircraft observed a red and white UAS at 1,000...","['RWY18L', 'RWY18C']",NaN,NaN,NaN,NaN
4,4,"Aircraft observed a black UAS at 1,000 feet wh...",['RWY 32R'],PAM,PAM,RWY 32R,['3 NM']
...,...,...,...,...,...,...,...
594,596,Aircraft reported a fixed wing UAS pass within...,['RWY 10'],SAV,SAV,RWY 10,['1 NM']
595,597,Aircraft reported a grey quad-copter UAS off t...,['RWY 27L'],SFB,SFB,RWY 27L,NaN
596,598,Aircraft reported a black quad-copter UAS off ...,['RWY 3R'],NaN,NaN,NaN,NaN
597,599,Aircraft reported a NMAC with a UAS while on a...,['RWY05'],NaN,NaN,NaN,['6 NM']


In [4]:
# Drop all NaN from rwy599_df
rwy599_df1 = rwy599_df.dropna().reset_index(drop=True)
rwy599_df1

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance
0,4,"Aircraft observed a black UAS at 1,000 feet wh...",['RWY 32R'],PAM,PAM,RWY 32R,['3 NM']
1,85,Aircraft observed a UAS above while W bound at...,['RWY 18R'],DFW,DFW,RWY 18R,['5 NM']
2,109,Aircraft observed a UAS at the same altitude w...,['RWY 36L'],MEM,MEM,RWY 36L,['6 NM']
3,124,Aircraft observed a red UAS off the right side...,['RWY 16R'],SEA,SEA,RWY 16R,['4 NM']
4,145,Aircraft observed a small red UAS E bound off ...,['RWY 15'],ADS,ADS,RWY 15,['3 NM']
...,...,...,...,...,...,...,...
97,516,Aircraft reported a UAS passing 100 feet below...,['RWY 27'],TLH,TLH,RWY 27,['2 NM']
98,588,"Aircraft reported a UAS off the right side, ap...",['RWY 25L'],LAX,LAX,RWY 25L,['3 NM']
99,591,"Aircraft reported a grey, basketball size, UAS...",['RWY 25L'],LAX,LAX,RWY 25L,['10 NM']
100,594,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RDU,RWY 23R,['3 NM']


In [5]:
# CHECK THAT THERE ARE NO DUPLICATE REMARKS
#rwy599_df1.drop_duplicates(subset=['REMARKS']) 

# THERE ARE STILL 102 ROWS SO WE ARE GOOD TO KEEP GOING

#### Import airports_runways_linked.csv dataset 

`Contains the cleaned and linked airports_cleaned.csv and rwy.csv supplemental datasets`

In [6]:
air_rwy = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_runways_linked.csv')
#air_rwy

#### Merge other_rwy_totalinfo.csv and airports_runways_linked.csv

`Merge the two datasets on the 'IDENT' field.`

In [7]:
rwy_linked = pd.merge(rwy599_df1, air_rwy, on = 'IDENT', how = 'left')
rwy_linked.rename(columns = {'properties.DESIGNATOR': 'DESIGNATOR'}, inplace = True)

In [8]:
rwy_linked

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance,properties.GLOBAL_ID,properties.AIRPORT_ID,DESIGNATOR,Runway.Latitude,Runway.Longitude,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,4,"Aircraft observed a black UAS at 1,000 feet wh...",['RWY 32R'],PAM,PAM,RWY 32R,['3 NM'],8FAB3563-C164-4D2A-A7D7-4CEDC40E1B5D,9907F7DB-7C4F-4CB6-98FA-F0462EE27DE9,14L/32R,-85.570825,30.068523,Tyndall AFB,30.069047,-85.574981,KPAM
1,4,"Aircraft observed a black UAS at 1,000 feet wh...",['RWY 32R'],PAM,PAM,RWY 32R,['3 NM'],EC5EA23A-FB43-4AF4-9F3A-6B72C1495F0C,9907F7DB-7C4F-4CB6-98FA-F0462EE27DE9,14R/32L,-85.579142,30.069627,Tyndall AFB,30.069047,-85.574981,KPAM
2,85,Aircraft observed a UAS above while W bound at...,['RWY 18R'],DFW,DFW,RWY 18R,['5 NM'],D60C0EEA-7524-4790-A294-CE2112A0E2FB,B371E693-B91F-40ED-ABB1-628EFE9D3111,13L/31R,-97.013278,32.905492,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW
3,85,Aircraft observed a UAS above while W bound at...,['RWY 18R'],DFW,DFW,RWY 18R,['5 NM'],A4F0B104-48EA-445F-8DAB-850EE0FCBA8C,B371E693-B91F-40ED-ABB1-628EFE9D3111,13R/31L,-97.075234,32.901832,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW
4,85,Aircraft observed a UAS above while W bound at...,['RWY 18R'],DFW,DFW,RWY 18R,['5 NM'],A088B20F-FBE9-4C4F-9937-F7ABADEE30CC,B371E693-B91F-40ED-ABB1-628EFE9D3111,17C/35C,-97.026110,32.900981,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,594,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RDU,RWY 23R,['3 NM'],91541445-E5CF-40C1-8E8F-3E759B9CA482,CB0BC5B9-0B40-44DE-96E3-5B8E2AEA35C4,05L/23R,-78.792358,35.882177,Raleigh-Durham Intl,35.877639,-78.787472,KRDU
325,594,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RDU,RWY 23R,['3 NM'],D1A26CC8-C4DA-48CF-A0B0-7FD883EBA2FA,CB0BC5B9-0B40-44DE-96E3-5B8E2AEA35C4,05R/23L,-78.790127,35.870421,Raleigh-Durham Intl,35.877639,-78.787472,KRDU
326,594,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RDU,RWY 23R,['3 NM'],B270C60D-5C0C-41EA-8410-5DA7D520B4E3,CB0BC5B9-0B40-44DE-96E3-5B8E2AEA35C4,14/32,-78.777558,35.870885,Raleigh-Durham Intl,35.877639,-78.787472,KRDU
327,596,Aircraft reported a fixed wing UAS pass within...,['RWY 10'],SAV,SAV,RWY 10,['1 NM'],B2D0E0C7-E412-458E-B683-52C4DD4E4EA9,D339C530-1D36-4587-8130-9E9938BF474D,01/19,-81.200005,32.124275,Savannah/Hilton Head Intl,32.127583,-81.202139,KSAV


#### Extract Runway Designator from RWYLOCATION field

`We want to be able to obtain the correct latitude/longitude coordinates for the appropriate runway mentioned in the remark.`

In [9]:
rwydesignator = []

for i in range(len(rwy_linked)):
    rwy_split = rwy_linked['RWYLOCATION'][i].split(' ')
    rwydesignator.append(rwy_split[1])

# Add split column with RWY designators to dataframe as new field
rwy_linked['RWYDESIGNATOR'] = rwydesignator

In [10]:
#rwy_linked

#### Check if RWY designator found in REMARK matches one of the RWY designators from the rwy.csv dataset

`If YES: RWYinREMARK field is 'True'`

`If NO: RWYinREMARK field is 'False'`
***
***NOTE:*** Since we linked on the 'IDENT' field, there are repeated remarks as a result of there being multiple runways at
each airport. Thus, we are only interested in the records for which the 'RWYinREMARK' field is 'True'.

In [11]:
rwy_linked['RWYinREMARK'] = rwy_linked.apply(lambda x: str(x.RWYDESIGNATOR) in str(x.DESIGNATOR), axis=1)

# Only keep records for which the RWY Designator is present in the DESIGNATOR field 
rwy95 = rwy_linked[rwy_linked['RWYinREMARK'] == True]
rwy95

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance,properties.GLOBAL_ID,properties.AIRPORT_ID,DESIGNATOR,Runway.Latitude,Runway.Longitude,NAME,LATITUDE,LONGITUDE,ICAO_ID,RWYDESIGNATOR,RWYinREMARK
0,4,"Aircraft observed a black UAS at 1,000 feet wh...",['RWY 32R'],PAM,PAM,RWY 32R,['3 NM'],8FAB3563-C164-4D2A-A7D7-4CEDC40E1B5D,9907F7DB-7C4F-4CB6-98FA-F0462EE27DE9,14L/32R,-85.570825,30.068523,Tyndall AFB,30.069047,-85.574981,KPAM,32R,True
8,85,Aircraft observed a UAS above while W bound at...,['RWY 18R'],DFW,DFW,RWY 18R,['5 NM'],ED9E6106-EB1D-401C-BCAE-E7A73A60A41C,B371E693-B91F-40ED-ABB1-628EFE9D3111,18R/36L,-97.054815,32.893723,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW,18R,True
12,109,Aircraft observed a UAS at the same altitude w...,['RWY 36L'],MEM,MEM,RWY 36L,['6 NM'],DC9631DF-1821-449A-AD41-8EA0B8B40375,DA96A371-EB42-4312-8EAF-FA237DE1E404,18R/36L,-89.987278,35.039251,Memphis Intl,35.042411,-89.976679,KMEM,36L,True
15,124,Aircraft observed a red UAS off the right side...,['RWY 16R'],SEA,SEA,RWY 16R,['4 NM'],D8DB5055-844E-44D9-A4DA-C7E399892BE1,5338FBAA-B183-4BC0-B3F1-27ABF6D1AD7C,16R/34L,-122.317932,47.449860,Seattle-Tacoma Intl,47.449889,-122.311778,KSEA,16R,True
16,145,Aircraft observed a small red UAS E bound off ...,['RWY 15'],ADS,ADS,RWY 15,['3 NM'],202E6282-A103-44B0-ACCE-3D984B66D091,3646423B-964B-466A-9CE9-3CB43457F772,15/33,-96.837304,32.970408,Addison,32.968556,-96.836444,KADS,15,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,516,Aircraft reported a UAS passing 100 feet below...,['RWY 27'],TLH,TLH,RWY 27,['2 NM'],6AFB0072-D2C9-4832-9B3A-9BA008C1E9F9,265A7B00-46FF-4744-8673-499589FFA2BF,09/27,-84.346397,30.391355,Tallahassee Intl,30.396754,-84.350869,KTLH,27,True
319,588,"Aircraft reported a UAS off the right side, ap...",['RWY 25L'],LAX,LAX,RWY 25L,['3 NM'],8ED22315-4327-432F-9C74-65E1A06D97A5,2C5866F1-5E27-4032-9AF8-988CE5DB82BA,07R/25L,-118.404501,33.935085,Los Angeles Intl,33.942496,-118.408049,KLAX,25L,True
323,591,"Aircraft reported a grey, basketball size, UAS...",['RWY 25L'],LAX,LAX,RWY 25L,['10 NM'],8ED22315-4327-432F-9C74-65E1A06D97A5,2C5866F1-5E27-4032-9AF8-988CE5DB82BA,07R/25L,-118.404501,33.935085,Los Angeles Intl,33.942496,-118.408049,KLAX,25L,True
324,594,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RDU,RWY 23R,['3 NM'],91541445-E5CF-40C1-8E8F-3E759B9CA482,CB0BC5B9-0B40-44DE-96E3-5B8E2AEA35C4,05L/23R,-78.792358,35.882177,Raleigh-Durham Intl,35.877639,-78.787472,KRDU,23R,True


#### Import airports_cleaned_declination.csv dataset 

`Contains the cleaned and linked airports_cleaned.csv and declination.csv supplemental datasets`

In [12]:
air_dec = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_cleaned_declination.csv')
air_dec

,Column1,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID,DECLINATION
0,0,6TS0,TRUE,34.089220,-101.740461,NaN,5.74941
1,1,WY47,TRUE,42.853576,-106.336415,NaN,8.90491
2,2,WA08,Zwainz Farms,47.713439,-117.919989,NaN,14.44256
3,3,9IN7,Zupancic Fld,39.354722,-86.306944,NaN,-4.69381
4,4,1TE4,Zuehl,29.495509,-98.157787,NaN,3.68769
...,...,...,...,...,...,...,...
19889,19889,55XA,1101 East Blvd Deer Park,29.702772,-95.101639,NaN,1.95632
19890,19890,WA54,1001 Fourth Avenue Plaza,47.606765,-122.333737,NaN,15.44604
19891,19891,FA10,1000 Museum,25.784228,-80.190017,NaN,-6.97982
19892,19892,TN41,100 Aker Wood,35.772778,-84.765278,NaN,-5.33458


#### Merge air_dec with rwy95

`Merge the two datasets on the 'IDENT' field.`

`air_dec --> Contains the cleaned airports supplemental dataset with the Declination value at the airport`

`rwy95 --> Dataframe containing 95 records referencing Runways that contain all relevant information (from the 599 OTHER records)`

In [13]:
rwy95_dec = pd.merge(rwy95, air_dec, on = 'IDENT', how = 'left')

# Only keep necessary columns/fields of interest
rwy95_decDF = pd.DataFrame(rwy95_dec[['REMARKS', 'UASLOCATION', 'IDENT', 'RWYLOCATION', 'UAS_Distance', 'RWYDESIGNATOR', 'Runway.Latitude', 'Runway.Longitude', 'DECLINATION']])

In [14]:
rwy95_decDF

,REMARKS,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance,RWYDESIGNATOR,Runway.Latitude,Runway.Longitude,DECLINATION
0,"Aircraft observed a black UAS at 1,000 feet wh...",PAM,PAM,RWY 32R,['3 NM'],32R,-85.570825,30.068523,-4.13012
1,Aircraft observed a UAS above while W bound at...,DFW,DFW,RWY 18R,['5 NM'],18R,-97.054815,32.893723,3.06900
2,Aircraft observed a UAS at the same altitude w...,MEM,MEM,RWY 36L,['6 NM'],36L,-89.987278,35.039251,-1.62738
3,Aircraft observed a red UAS off the right side...,SEA,SEA,RWY 16R,['4 NM'],16R,-122.317932,47.449860,15.40471
4,Aircraft observed a small red UAS E bound off ...,ADS,ADS,RWY 15,['3 NM'],15,-96.837304,32.970408,2.94849
...,...,...,...,...,...,...,...,...,...
90,Aircraft reported a UAS passing 100 feet below...,TLH,TLH,RWY 27,['2 NM'],27,-84.346397,30.391355,-4.96630
91,"Aircraft reported a UAS off the right side, ap...",LAX,LAX,RWY 25L,['3 NM'],25L,-118.404501,33.935085,11.71212
92,"Aircraft reported a grey, basketball size, UAS...",LAX,LAX,RWY 25L,['10 NM'],25L,-118.404501,33.935085,11.71212
93,Aircraft reported a UAS off the left side whil...,RDU,RDU,RWY 23R,['3 NM'],23R,-78.792358,35.882177,-9.22834


### Calculate Bearing Information for Runways

`To do this, you take the number portion of the Runway Designator (in the 'RWYDESIGNATOR' field) and multiply it by 10 (effectively adding a 0 to the end) and add the DECLINATION to this number`

***Example:*** *RWY 32R has designator 32R which indicates a bearing of 320 degrees*

In [15]:
designator = rwy95_decDF['RWYDESIGNATOR']
designator_bearing = []

for i in range(len(rwy95_decDF)):
    designator_bearing.append(int(re.sub("\D","",designator[i]))*10 + rwy95_decDF['DECLINATION'][i])

rwy95_decDF['RWY_BEARING'] = designator_bearing

In [16]:
rwy95_decDF

,REMARKS,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance,RWYDESIGNATOR,Runway.Latitude,Runway.Longitude,DECLINATION,RWY_BEARING
0,"Aircraft observed a black UAS at 1,000 feet wh...",PAM,PAM,RWY 32R,['3 NM'],32R,-85.570825,30.068523,-4.13012,315.86988
1,Aircraft observed a UAS above while W bound at...,DFW,DFW,RWY 18R,['5 NM'],18R,-97.054815,32.893723,3.06900,183.06900
2,Aircraft observed a UAS at the same altitude w...,MEM,MEM,RWY 36L,['6 NM'],36L,-89.987278,35.039251,-1.62738,358.37262
3,Aircraft observed a red UAS off the right side...,SEA,SEA,RWY 16R,['4 NM'],16R,-122.317932,47.449860,15.40471,175.40471
4,Aircraft observed a small red UAS E bound off ...,ADS,ADS,RWY 15,['3 NM'],15,-96.837304,32.970408,2.94849,152.94849
...,...,...,...,...,...,...,...,...,...,...
90,Aircraft reported a UAS passing 100 feet below...,TLH,TLH,RWY 27,['2 NM'],27,-84.346397,30.391355,-4.96630,265.03370
91,"Aircraft reported a UAS off the right side, ap...",LAX,LAX,RWY 25L,['3 NM'],25L,-118.404501,33.935085,11.71212,261.71212
92,"Aircraft reported a grey, basketball size, UAS...",LAX,LAX,RWY 25L,['10 NM'],25L,-118.404501,33.935085,11.71212,261.71212
93,Aircraft reported a UAS off the left side whil...,RDU,RDU,RWY 23R,['3 NM'],23R,-78.792358,35.882177,-9.22834,220.77166


### Convert Distances from NM to Kilometers

In [17]:
# Convert all distance from NM to kilometers
dist_kilo = []

for i in range(len(rwy95_decDF)):
    distanceKilo = int(re.sub("\D","",rwy95_decDF['UAS_Distance'][i]))* 1.852 # converting NM to kilometers
    dist_kilo.append(distanceKilo)
    
rwy95_decDF['Distance_Kilometers'] = dist_kilo

In [18]:
rwy95_decDF

,REMARKS,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance,RWYDESIGNATOR,Runway.Latitude,Runway.Longitude,DECLINATION,RWY_BEARING,Distance_Kilometers
0,"Aircraft observed a black UAS at 1,000 feet wh...",PAM,PAM,RWY 32R,['3 NM'],32R,-85.570825,30.068523,-4.13012,315.86988,5.556
1,Aircraft observed a UAS above while W bound at...,DFW,DFW,RWY 18R,['5 NM'],18R,-97.054815,32.893723,3.06900,183.06900,9.260
2,Aircraft observed a UAS at the same altitude w...,MEM,MEM,RWY 36L,['6 NM'],36L,-89.987278,35.039251,-1.62738,358.37262,11.112
3,Aircraft observed a red UAS off the right side...,SEA,SEA,RWY 16R,['4 NM'],16R,-122.317932,47.449860,15.40471,175.40471,7.408
4,Aircraft observed a small red UAS E bound off ...,ADS,ADS,RWY 15,['3 NM'],15,-96.837304,32.970408,2.94849,152.94849,5.556
...,...,...,...,...,...,...,...,...,...,...,...
90,Aircraft reported a UAS passing 100 feet below...,TLH,TLH,RWY 27,['2 NM'],27,-84.346397,30.391355,-4.96630,265.03370,3.704
91,"Aircraft reported a UAS off the right side, ap...",LAX,LAX,RWY 25L,['3 NM'],25L,-118.404501,33.935085,11.71212,261.71212,5.556
92,"Aircraft reported a grey, basketball size, UAS...",LAX,LAX,RWY 25L,['10 NM'],25L,-118.404501,33.935085,11.71212,261.71212,18.520
93,Aircraft reported a UAS off the left side whil...,RDU,RDU,RWY 23R,['3 NM'],23R,-78.792358,35.882177,-9.22834,220.77166,5.556


# SWITCH Runway.Latitude and Runway.Longitude (I THINK ERIC HAS THESE MIXED UP ACCIDENTALLY)

In [19]:
rwy95_decDF.rename(columns = {'Runway.Latitude': 'Runway_Longitude', 'Runway.Longitude' : 'Runway_Latitude'}, inplace = True)

### Calculate UAS Latitude/Longitude Information using geopy library

In [20]:
uas_lat = []
uas_long = []


for i in range(len(rwy95_decDF)):
    lat_rwy = pd.to_numeric(rwy95_decDF['Runway_Latitude'][i])
    long_rwy = pd.to_numeric(rwy95_decDF['Runway_Longitude'][i])
    b = pd.to_numeric(rwy95_decDF['RWY_BEARING'][i])
    d = pd.to_numeric(rwy95_decDF['Distance_Kilometers'][i])

    origin = geopy.Point(lat_rwy, long_rwy)
    destination = geodesic(kilometers=d).destination(origin,b)

    lat2, lon2, = destination.latitude, destination.longitude

    uas_lat.append(lat2)
    uas_long.append(lon2)

# Append UAS Lat/Long information to DataFrame
rwy95_decDF['UAS_Latitude'] = uas_lat
rwy95_decDF['UAS_Longitude'] = uas_long

In [21]:
rwy95_decDF

,REMARKS,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance,RWYDESIGNATOR,Runway_Longitude,Runway_Latitude,DECLINATION,RWY_BEARING,Distance_Kilometers,UAS_Latitude,UAS_Longitude
0,"Aircraft observed a black UAS at 1,000 feet wh...",PAM,PAM,RWY 32R,['3 NM'],32R,-85.570825,30.068523,-4.13012,315.86988,5.556,30.104491,-85.610962
1,Aircraft observed a UAS above while W bound at...,DFW,DFW,RWY 18R,['5 NM'],18R,-97.054815,32.893723,3.06900,183.06900,9.260,32.810345,-97.060108
2,Aircraft observed a UAS at the same altitude w...,MEM,MEM,RWY 36L,['6 NM'],36L,-89.987278,35.039251,-1.62738,358.37262,11.112,35.139371,-89.990741
3,Aircraft observed a red UAS off the right side...,SEA,SEA,RWY 16R,['4 NM'],16R,-122.317932,47.449860,15.40471,175.40471,7.408,47.383443,-122.310072
4,Aircraft observed a small red UAS E bound off ...,ADS,ADS,RWY 15,['3 NM'],15,-96.837304,32.970408,2.94849,152.94849,5.556,32.925788,-96.810288
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Aircraft reported a UAS passing 100 feet below...,TLH,TLH,RWY 27,['2 NM'],27,-84.346397,30.391355,-4.96630,265.03370,3.704,30.388457,-84.384793
91,"Aircraft reported a UAS off the right side, ap...",LAX,LAX,RWY 25L,['3 NM'],25L,-118.404501,33.935085,11.71212,261.71212,5.556,33.927851,-118.463962
92,"Aircraft reported a grey, basketball size, UAS...",LAX,LAX,RWY 25L,['10 NM'],25L,-118.404501,33.935085,11.71212,261.71212,18.520,33.910858,-118.602666
93,Aircraft reported a UAS off the left side whil...,RDU,RDU,RWY 23R,['3 NM'],23R,-78.792358,35.882177,-9.22834,220.77166,5.556,35.844248,-78.832521


In [22]:
# Export final new_df to .csv 
rwy95_decDF.to_csv('other2142_runways95_UAS_latlong.csv', index = False)

***
# Exploring the 504 Remaining Remarks Referencing Runways (RWYS)
***
`These Remarks are a subset of the 2142 records that were NOT hit on by the initial regex: 'XX NM dir XXX'.`

In [23]:
rwy_unid504 = pd.DataFrame(rwy599, index = set(rwy599.index).difference(set(rwy95_decDF.index))).reset_index(drop=True)
rwy_unid504

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance
0,95,Aircraft observed a white UAS at 500 feet whil...,['RWY 9L'],NaN,NaN,NaN,[]
1,96,Aircraft observed a UAS off the right side whi...,['RWY 24R'],NaN,NaN,NaN,['5NM']
2,97,Aircraft observed a flight of 2 white UAS on t...,['RWY 24'],CRQ,CRQ,RWY 24,[]
3,98,Aircraft observed a black Quad-copter UAS off ...,['RWY17L'],NaN,NaN,NaN,[]
4,99,Aircraft observed a white UAS off the left sid...,['RWY 8L'],NaN,NaN,NaN,['3NM']
...,...,...,...,...,...,...,...
499,596,Aircraft reported a fixed wing UAS pass within...,['RWY 10'],SAV,SAV,RWY 10,['1 NM']
500,597,Aircraft reported a grey quad-copter UAS off t...,['RWY 27L'],SFB,SFB,RWY 27L,[]
501,598,Aircraft reported a black quad-copter UAS off ...,['RWY 3R'],NaN,NaN,NaN,[]
502,599,Aircraft reported a NMAC with a UAS while on a...,['RWY05'],NaN,NaN,NaN,['6 NM']


# Look at the records for which there is a valid IDENT

`There are 202 records (within the 504 unidentified runway remarks) that have valid IDENT/REPORTINGFACILITY Information associated with the Runway mentioned within the Remark.`

In [24]:
rwy202 = rwy_unid504[rwy_unid504['IDENT'].notnull()].reset_index(drop=True)
rwy202

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance
0,97,Aircraft observed a flight of 2 white UAS on t...,['RWY 24'],CRQ,CRQ,RWY 24,[]
1,104,Aircraft observed a UAS off the left side whil...,['RWY 23L'],RDU,RDU,RWY 23L,[]
2,107,Aircraft observed a UAS while NE bound at 900 ...,['RWY 28'],MYF,MYF,RWY 28,[]
3,109,Aircraft observed a UAS at the same altitude w...,['RWY 36L'],MEM,MEM,RWY 36L,['6 NM']
4,111,"Aircraft observed a UAS while S bound at 1,000...",['RWY 26R'],LGB,LGB,RWY 26R,[]
...,...,...,...,...,...,...,...
197,593,"Aircraft reported a UAS at 4,400 feet while on...",['RWY 35C'],DFW,DFW,RWY 35C,[]
198,594,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RDU,RWY 23R,['3 NM']
199,596,Aircraft reported a fixed wing UAS pass within...,['RWY 10'],SAV,SAV,RWY 10,['1 NM']
200,597,Aircraft reported a grey quad-copter UAS off t...,['RWY 27L'],SFB,SFB,RWY 27L,[]


In [25]:
# First we need to replace all [] in UAS_Distance field with NaN
rwy202_df = rwy202.mask(rwy202.applymap(str).eq('[]'))
#rwy202_df

## Explore the records for which there is valid UAS_Distance (in NM) information associated with the RWY Location

`There are 100 records (of the 202 with valid Reporting Facility / IDENT information) that also contain a valid UAS Distance (in NM).`

In [26]:
rwy202_dist = rwy202_df[rwy202_df['UAS_Distance'].notnull()].reset_index(drop=True)
rwy202_dist

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance
0,109,Aircraft observed a UAS at the same altitude w...,['RWY 36L'],MEM,MEM,RWY 36L,['6 NM']
1,124,Aircraft observed a red UAS off the right side...,['RWY 16R'],SEA,SEA,RWY 16R,['4 NM']
2,145,Aircraft observed a small red UAS E bound off ...,['RWY 15'],ADS,ADS,RWY 15,['3 NM']
3,148,"Aircraft observed a UAS, operating at approxim...",['RWY 10L'],PDX,PDX,RWY 10L,['7 NM']
4,160,Aircraft observed a UAS 500 feet below the air...,['RWY 25L'],LAX,LAX,RWY 25L,['2 NM']
...,...,...,...,...,...,...,...
95,516,Aircraft reported a UAS passing 100 feet below...,['RWY 27'],TLH,TLH,RWY 27,['2 NM']
96,588,"Aircraft reported a UAS off the right side, ap...",['RWY 25L'],LAX,LAX,RWY 25L,['3 NM']
97,591,"Aircraft reported a grey, basketball size, UAS...",['RWY 25L'],LAX,LAX,RWY 25L,['10 NM']
98,594,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RDU,RWY 23R,['3 NM']


In [27]:
# Export to .csv 
rwy202_dist.to_csv('rwy100.csv', index = False)

#### Merge rwy202_dist and airports_runways_linked.csv (air_rwy)

`Merge the two datasets on the 'IDENT' field.`

In [28]:
rwy100_airwy = pd.merge(rwy202_dist, air_rwy, on = 'IDENT', how = 'left').reset_index(drop=True)
rwy100_airwy.rename(columns = {'properties.DESIGNATOR': 'DESIGNATOR'}, inplace = True)
rwy100_airwy

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance,properties.GLOBAL_ID,properties.AIRPORT_ID,DESIGNATOR,Runway.Latitude,Runway.Longitude,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,109,Aircraft observed a UAS at the same altitude w...,['RWY 36L'],MEM,MEM,RWY 36L,['6 NM'],3FD784E7-DFCC-4115-842D-60DB410A9D25,DA96A371-EB42-4312-8EAF-FA237DE1E404,09/27,-89.973792,35.058252,Memphis Intl,35.042411,-89.976679,KMEM
1,109,Aircraft observed a UAS at the same altitude w...,['RWY 36L'],MEM,MEM,RWY 36L,['6 NM'],462D6D8A-8D57-443B-B452-E01E8E40D249,DA96A371-EB42-4312-8EAF-FA237DE1E404,18C/36C,-89.975969,35.042383,Memphis Intl,35.042411,-89.976679,KMEM
2,109,Aircraft observed a UAS at the same altitude w...,['RWY 36L'],MEM,MEM,RWY 36L,['6 NM'],DE9A1353-6BD4-410A-A407-B2A253445358,DA96A371-EB42-4312-8EAF-FA237DE1E404,18L/36R,-89.972799,35.038933,Memphis Intl,35.042411,-89.976679,KMEM
3,109,Aircraft observed a UAS at the same altitude w...,['RWY 36L'],MEM,MEM,RWY 36L,['6 NM'],DC9631DF-1821-449A-AD41-8EA0B8B40375,DA96A371-EB42-4312-8EAF-FA237DE1E404,18R/36L,-89.987278,35.039251,Memphis Intl,35.042411,-89.976679,KMEM
4,124,Aircraft observed a red UAS off the right side...,['RWY 16R'],SEA,SEA,RWY 16R,['4 NM'],7AC9CEA7-9C80-44BD-84DC-63EDFA5C4EC2,5338FBAA-B183-4BC0-B3F1-27ABF6D1AD7C,16C/34C,-122.311074,47.448312,Seattle-Tacoma Intl,47.449889,-122.311778,KSEA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,594,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RDU,RWY 23R,['3 NM'],91541445-E5CF-40C1-8E8F-3E759B9CA482,CB0BC5B9-0B40-44DE-96E3-5B8E2AEA35C4,05L/23R,-78.792358,35.882177,Raleigh-Durham Intl,35.877639,-78.787472,KRDU
316,594,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RDU,RWY 23R,['3 NM'],D1A26CC8-C4DA-48CF-A0B0-7FD883EBA2FA,CB0BC5B9-0B40-44DE-96E3-5B8E2AEA35C4,05R/23L,-78.790127,35.870421,Raleigh-Durham Intl,35.877639,-78.787472,KRDU
317,594,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RDU,RWY 23R,['3 NM'],B270C60D-5C0C-41EA-8410-5DA7D520B4E3,CB0BC5B9-0B40-44DE-96E3-5B8E2AEA35C4,14/32,-78.777558,35.870885,Raleigh-Durham Intl,35.877639,-78.787472,KRDU
318,596,Aircraft reported a fixed wing UAS pass within...,['RWY 10'],SAV,SAV,RWY 10,['1 NM'],B2D0E0C7-E412-458E-B683-52C4DD4E4EA9,D339C530-1D36-4587-8130-9E9938BF474D,01/19,-81.200005,32.124275,Savannah/Hilton Head Intl,32.127583,-81.202139,KSAV


#### Extract Runway Designator from RWYLOCATION field

`We want to be able to obtain the correct latitude/longitude coordinates for the appropriate runway mentioned in the remark.`

In [29]:
rwydesignator = []

for i in range(len(rwy100_airwy)):
    rwy_split = rwy100_airwy['RWYLOCATION'][i].split(' ')
    rwydesignator.append(rwy_split[1])

# Add split column with RWY designators to dataframe as new field
rwy100_airwy['RWYDESIGNATOR'] = rwydesignator

#### Check if RWY designator found in REMARK matches one of the RWY designators from the rwy.csv dataset

`If YES: RWYinREMARK field is 'True'`

`If NO: RWYinREMARK field is 'False'`
***
***NOTE:*** Since we linked on the 'IDENT' field, there are repeated remarks as a result of there being multiple runways at
each airport. Thus, we are only interested in the records for which the 'RWYinREMARK' field is 'True'.

In [30]:
rwy100_airwy['RWYinREMARK'] = rwy100_airwy.apply(lambda x: str(x.RWYDESIGNATOR) in str(x.DESIGNATOR), axis=1)

# Only keep records for which the RWY Designator is present in the DESIGNATOR field 
rwy93 = rwy100_airwy[rwy100_airwy['RWYinREMARK'] == True].reset_index(drop=True)

In [31]:
rwy93

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance,properties.GLOBAL_ID,properties.AIRPORT_ID,DESIGNATOR,Runway.Latitude,Runway.Longitude,NAME,LATITUDE,LONGITUDE,ICAO_ID,RWYDESIGNATOR,RWYinREMARK
0,109,Aircraft observed a UAS at the same altitude w...,['RWY 36L'],MEM,MEM,RWY 36L,['6 NM'],DC9631DF-1821-449A-AD41-8EA0B8B40375,DA96A371-EB42-4312-8EAF-FA237DE1E404,18R/36L,-89.987278,35.039251,Memphis Intl,35.042411,-89.976679,KMEM,36L,True
1,124,Aircraft observed a red UAS off the right side...,['RWY 16R'],SEA,SEA,RWY 16R,['4 NM'],D8DB5055-844E-44D9-A4DA-C7E399892BE1,5338FBAA-B183-4BC0-B3F1-27ABF6D1AD7C,16R/34L,-122.317932,47.449860,Seattle-Tacoma Intl,47.449889,-122.311778,KSEA,16R,True
2,145,Aircraft observed a small red UAS E bound off ...,['RWY 15'],ADS,ADS,RWY 15,['3 NM'],202E6282-A103-44B0-ACCE-3D984B66D091,3646423B-964B-466A-9CE9-3CB43457F772,15/33,-96.837304,32.970408,Addison,32.968556,-96.836444,KADS,15,True
3,148,"Aircraft observed a UAS, operating at approxim...",['RWY 10L'],PDX,PDX,RWY 10L,['7 NM'],DCC8037A-6FE3-4A15-8A3B-3E7253D8FE19,3B44B4F3-705A-4A4D-AEBC-19BAF63DA508,10L/28R,-122.586637,45.591254,Portland Intl,45.588709,-122.596869,KPDX,10L,True
4,160,Aircraft observed a UAS 500 feet below the air...,['RWY 25L'],LAX,LAX,RWY 25L,['2 NM'],8ED22315-4327-432F-9C74-65E1A06D97A5,2C5866F1-5E27-4032-9AF8-988CE5DB82BA,07R/25L,-118.404501,33.935085,Los Angeles Intl,33.942496,-118.408049,KLAX,25L,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,516,Aircraft reported a UAS passing 100 feet below...,['RWY 27'],TLH,TLH,RWY 27,['2 NM'],6AFB0072-D2C9-4832-9B3A-9BA008C1E9F9,265A7B00-46FF-4744-8673-499589FFA2BF,09/27,-84.346397,30.391355,Tallahassee Intl,30.396754,-84.350869,KTLH,27,True
89,588,"Aircraft reported a UAS off the right side, ap...",['RWY 25L'],LAX,LAX,RWY 25L,['3 NM'],8ED22315-4327-432F-9C74-65E1A06D97A5,2C5866F1-5E27-4032-9AF8-988CE5DB82BA,07R/25L,-118.404501,33.935085,Los Angeles Intl,33.942496,-118.408049,KLAX,25L,True
90,591,"Aircraft reported a grey, basketball size, UAS...",['RWY 25L'],LAX,LAX,RWY 25L,['10 NM'],8ED22315-4327-432F-9C74-65E1A06D97A5,2C5866F1-5E27-4032-9AF8-988CE5DB82BA,07R/25L,-118.404501,33.935085,Los Angeles Intl,33.942496,-118.408049,KLAX,25L,True
91,594,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RDU,RWY 23R,['3 NM'],91541445-E5CF-40C1-8E8F-3E759B9CA482,CB0BC5B9-0B40-44DE-96E3-5B8E2AEA35C4,05L/23R,-78.792358,35.882177,Raleigh-Durham Intl,35.877639,-78.787472,KRDU,23R,True


#### Merge air_dec with rwy93

`Merge the two datasets on the 'IDENT' field.`

`air_dec --> Contains the cleaned airports supplemental dataset with the Declination value at the airport`

`rwy93 --> Dataframe containing 93 records referencing Runways that contain all relevant information (from the 100 records containing UAS_Distance information)`

In [32]:
rwy93_dec = pd.merge(rwy93, air_dec, on = 'IDENT', how = 'left')

# Only keep necessary columns/fields of interest
rwy93_decDF = pd.DataFrame(rwy93_dec[['REMARKS', 'UASLOCATION', 'IDENT', 'RWYLOCATION', 'UAS_Distance', 'RWYDESIGNATOR', 'Runway.Latitude', 'Runway.Longitude', 'DECLINATION']])

### Calculate Bearing Information for Runways

`To do this, you take the number portion of the Runway Designator (in the 'RWYDESIGNATOR' field) and multiply it by 10 (effectively adding a 0 to the end) and add the DECLINATION to this number`

***Example:*** *RWY 32R has designator 32R which indicates a bearing of 320 degrees*

In [33]:
designator = rwy93_decDF['RWYDESIGNATOR']
designator_bearing = []

for i in range(len(rwy93_decDF)):
    designator_bearing.append(int(re.sub("\D","",designator[i]))*10 + rwy93_decDF['DECLINATION'][i])

rwy93_decDF['RWY_BEARING'] = designator_bearing

### Convert Distances from NM to Kilometers

In [34]:
# Convert all distance from NM to kilometers
dist_kilo = []

for i in range(len(rwy93_decDF)):
    distanceKilo = int(re.sub("\D","",rwy93_decDF['UAS_Distance'][i]))* 1.852 # converting NM to kilometers
    dist_kilo.append(distanceKilo)
    
rwy93_decDF['Distance_Kilometers'] = dist_kilo

In [35]:
rwy93_decDF.rename(columns = {'Runway.Latitude': 'Runway_Longitude', 'Runway.Longitude' : 'Runway_Latitude'}, inplace = True)

## Calculate UAS Latitude/Longitude Information using geopy library

In [36]:
uas_lat = []
uas_long = []


for i in range(len(rwy93_decDF)):
    lat_rwy = pd.to_numeric(rwy93_decDF['Runway_Latitude'][i])
    long_rwy = pd.to_numeric(rwy93_decDF['Runway_Longitude'][i])
    b = pd.to_numeric(rwy93_decDF['RWY_BEARING'][i])
    d = pd.to_numeric(rwy93_decDF['Distance_Kilometers'][i])

    origin = geopy.Point(lat_rwy, long_rwy)
    destination = geodesic(kilometers=d).destination(origin,b)

    lat2, lon2, = destination.latitude, destination.longitude

    uas_lat.append(lat2)
    uas_long.append(lon2)

# Append UAS Lat/Long information to DataFrame
rwy93_decDF['UAS_Latitude'] = uas_lat
rwy93_decDF['UAS_Longitude'] = uas_long

In [37]:
rwy93_decDF

,REMARKS,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance,RWYDESIGNATOR,Runway_Longitude,Runway_Latitude,DECLINATION,RWY_BEARING,Distance_Kilometers,UAS_Latitude,UAS_Longitude
0,Aircraft observed a UAS at the same altitude w...,MEM,MEM,RWY 36L,['6 NM'],36L,-89.987278,35.039251,-1.62738,358.37262,11.112,35.139371,-89.990741
1,Aircraft observed a red UAS off the right side...,SEA,SEA,RWY 16R,['4 NM'],16R,-122.317932,47.449860,15.40471,175.40471,7.408,47.383443,-122.310072
2,Aircraft observed a small red UAS E bound off ...,ADS,ADS,RWY 15,['3 NM'],15,-96.837304,32.970408,2.94849,152.94849,5.556,32.925788,-96.810288
3,"Aircraft observed a UAS, operating at approxim...",PDX,PDX,RWY 10L,['7 NM'],10L,-122.586637,45.591254,15.01885,115.01885,12.964,45.541825,-122.436221
4,Aircraft observed a UAS 500 feet below the air...,LAX,LAX,RWY 25L,['2 NM'],25L,-118.404501,33.935085,11.71212,261.71212,3.704,33.930265,-118.444143
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Aircraft reported a UAS passing 100 feet below...,TLH,TLH,RWY 27,['2 NM'],27,-84.346397,30.391355,-4.96630,265.03370,3.704,30.388457,-84.384793
89,"Aircraft reported a UAS off the right side, ap...",LAX,LAX,RWY 25L,['3 NM'],25L,-118.404501,33.935085,11.71212,261.71212,5.556,33.927851,-118.463962
90,"Aircraft reported a grey, basketball size, UAS...",LAX,LAX,RWY 25L,['10 NM'],25L,-118.404501,33.935085,11.71212,261.71212,18.520,33.910858,-118.602666
91,Aircraft reported a UAS off the left side whil...,RDU,RDU,RWY 23R,['3 NM'],23R,-78.792358,35.882177,-9.22834,220.77166,5.556,35.844248,-78.832521


In [38]:
# Export to .csv 
rwy93_decDF.to_csv('rwy93_uasLatLong.csv', index = False)

#### Obtain Dataframe (and corresponding .csv file) for the 7 records for which UAS Lat/Long information was not captured

In [39]:
rwy_unid7 = pd.DataFrame(rwy202_dist, index = set(rwy202_dist.index).difference(set(rwy93_decDF.index))).reset_index(drop=True)

In [40]:
rwy_unid7

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance
0,588,"Aircraft reported a UAS off the right side, ap...",['RWY 25L'],LAX,LAX,RWY 25L,['3 NM']
1,591,"Aircraft reported a grey, basketball size, UAS...",['RWY 25L'],LAX,LAX,RWY 25L,['10 NM']
2,594,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RDU,RWY 23R,['3 NM']
3,596,Aircraft reported a fixed wing UAS pass within...,['RWY 10'],SAV,SAV,RWY 10,['1 NM']
4,467,Aircraft reported a UAS off their left side at...,['RWY 31'],2 NM SE LGA,LGA,RWY 31,['2 NM']
5,511,"Aircraft reported a red fixed wing UAS, 200 fe...",['RWY 24'],WOC,WOC,RWY 24,['4 NM']
6,516,Aircraft reported a UAS passing 100 feet below...,['RWY 27'],TLH,TLH,RWY 27,['2 NM']


In [41]:
# Export to .csv 
rwy_unid7.to_csv('rwy7_EXCEPTIONS.csv', index = False)

***

## Explore the records for which there is NO valid UAS_Distance (in NM) information associated with the RWY Location

`There are 102 records (of the 202 with valid Reporting Facility / IDENT information) that DO NOT contain a valid UAS Distance (in NM).`

In [42]:
rwy202_noDist = rwy202_df[rwy202_df['UAS_Distance'].isnull()].reset_index(drop=True)
rwy202_noDist

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance
0,97,Aircraft observed a flight of 2 white UAS on t...,['RWY 24'],CRQ,CRQ,RWY 24,NaN
1,104,Aircraft observed a UAS off the left side whil...,['RWY 23L'],RDU,RDU,RWY 23L,NaN
2,107,Aircraft observed a UAS while NE bound at 900 ...,['RWY 28'],MYF,MYF,RWY 28,NaN
3,111,"Aircraft observed a UAS while S bound at 1,000...",['RWY 26R'],LGB,LGB,RWY 26R,NaN
4,112,Aircraft observed a UAS at the 2 o'clock posit...,['RWY 19R'],MKE,MKE,RWY 19R,NaN
...,...,...,...,...,...,...,...
97,589,Aircraft reported a red and white quad-copter ...,['RWY 15'],PNE,PNE,RWY 15,NaN
98,590,"Aircraft reported a UAS off left side, while o...",['RWY 35L'],DWH,DWH,RWY 35L,NaN
99,593,"Aircraft reported a UAS at 4,400 feet while on...",['RWY 35C'],DFW,DFW,RWY 35C,NaN
100,597,Aircraft reported a grey quad-copter UAS off t...,['RWY 27L'],SFB,SFB,RWY 27L,NaN


In [43]:
# Export to .csv 
rwy202_noDist.to_csv('rwy102.csv', index = False)

#### Check for Remarks that have substring 'XX mile final' before mention of a Runway

`Be sure to account for multiple types of XX : e.g. 1.5, 05, 14, 0.75, etc.`

In [44]:
# Create List that contains each Standard Remark, and the Heading/Direction
# information contained in each remark

remark_uas_loc = []
remarks = rwy202_noDist['REMARKS']

# regular expression for any heading/direction
headir_regex = '\.?[0-9]\.?[0-9]*[0-9]*\s?mile?[s]?\s?final?'

# Loop through all remarks and search for the heading/direction regex above
for i in range(len(remarks)):
    head_dir = re.findall(headir_regex, remarks[i])
    remark_uas_loc.append(remarks[i])
    remark_uas_loc.append(head_dir)

# Split Remarks and Heading/Directions into two seperate lists and create
# pandas dataframe
remark = []
uas_loc = []

for i in range(0, len(remark_uas_loc), 2):
    remark.append(remark_uas_loc[i])
    uas_loc.append(remark_uas_loc[i+1])

rwy202_noDist['UAS_Distance'] = uas_loc

In [45]:
rwy202_noDist

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance
0,97,Aircraft observed a flight of 2 white UAS on t...,['RWY 24'],CRQ,CRQ,RWY 24,[]
1,104,Aircraft observed a UAS off the left side whil...,['RWY 23L'],RDU,RDU,RWY 23L,[]
2,107,Aircraft observed a UAS while NE bound at 900 ...,['RWY 28'],MYF,MYF,RWY 28,[]
3,111,"Aircraft observed a UAS while S bound at 1,000...",['RWY 26R'],LGB,LGB,RWY 26R,[]
4,112,Aircraft observed a UAS at the 2 o'clock posit...,['RWY 19R'],MKE,MKE,RWY 19R,[1.5 mile final]
...,...,...,...,...,...,...,...
97,589,Aircraft reported a red and white quad-copter ...,['RWY 15'],PNE,PNE,RWY 15,[]
98,590,"Aircraft reported a UAS off left side, while o...",['RWY 35L'],DWH,DWH,RWY 35L,[]
99,593,"Aircraft reported a UAS at 4,400 feet while on...",['RWY 35C'],DFW,DFW,RWY 35C,[]
100,597,Aircraft reported a grey quad-copter UAS off t...,['RWY 27L'],SFB,SFB,RWY 27L,[]


`There are 12 records (of the 102 with No Distance information) that actually contain distance information, but in the format:
'XX mile final'`

In [46]:
# First we need to replace all [] in UAS_Distance field with NaN
rwy102_df = rwy202_noDist.mask(rwy202_noDist.applymap(str).eq('[]'))
rwy102_df_MF = rwy102_df[rwy102_df['UAS_Distance'].notnull()].reset_index(drop=True)
rwy102_df_MF

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance
0,112,Aircraft observed a UAS at the 2 o'clock posit...,['RWY 19R'],MKE,MKE,RWY 19R,[1.5 mile final]
1,132,Aircraft observed a silver quadcopter UAS fro...,['RWY 17R'],AUS,AUS,RWY 17R,[7 mile final]
2,157,Aircraft observed a UAS SW bound at 900 feet o...,['RWY 22'],LGA,LGA,RWY 22,[2.5 mile final]
3,162,Aircraft observed a silver UAS on 4 mile final...,['RWY 31R'],JFK,JFK,RWY 31R,[4 mile final]
4,337,Aircraft observed a UAS while descending throu...,['RWY 24'],ACK,ACK,NaN,[2 mile final]
5,338,Aircraft observed a UAS while descending throu...,['RWY 24'],ACK,ACK,NaN,[2 mile final]
6,359,Aircraft observed a UAS 800 feet overhead off ...,['RWY 13'],SGJ,SGJ,RWY 13,[1 mile final]
7,364,Aircraft observed a black and silver UAS from ...,['RWY 25L'],LAX,LAX,RWY 25L,[2 mile final]
8,366,"Aircraft observed a UAS off the left at 2,400 ...",['RWY 35L'],LAX,LAX,RWY 35L,[7 mile final]
9,376,Aircraft observed a white UAS off the left sid...,['RWY 25L'],LAX,LAX,RWY 25L,[3 mile final]


#### Convert Distances from NM to Kilometers

`Using the 'UAS_Distnace' field from rwy102_df_MF`

In [47]:
# Convert all distance from NM to kilometers
dist_kilo = []

for i in range(len(rwy102_df_MF)):
    distanceKilo = float(re.sub(" mile final","",rwy102_df_MF['UAS_Distance'][i][0]))* 1.852 # converting NM to kilometers
    dist_kilo.append(distanceKilo)
    
rwy102_df_MF['Distance_Kilometers'] = dist_kilo
rwy102_df_MF

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance,Distance_Kilometers
0,112,Aircraft observed a UAS at the 2 o'clock posit...,['RWY 19R'],MKE,MKE,RWY 19R,[1.5 mile final],2.778
1,132,Aircraft observed a silver quadcopter UAS fro...,['RWY 17R'],AUS,AUS,RWY 17R,[7 mile final],12.964
2,157,Aircraft observed a UAS SW bound at 900 feet o...,['RWY 22'],LGA,LGA,RWY 22,[2.5 mile final],4.630
3,162,Aircraft observed a silver UAS on 4 mile final...,['RWY 31R'],JFK,JFK,RWY 31R,[4 mile final],7.408
4,337,Aircraft observed a UAS while descending throu...,['RWY 24'],ACK,ACK,NaN,[2 mile final],3.704
5,338,Aircraft observed a UAS while descending throu...,['RWY 24'],ACK,ACK,NaN,[2 mile final],3.704
6,359,Aircraft observed a UAS 800 feet overhead off ...,['RWY 13'],SGJ,SGJ,RWY 13,[1 mile final],1.852
7,364,Aircraft observed a black and silver UAS from ...,['RWY 25L'],LAX,LAX,RWY 25L,[2 mile final],3.704
8,366,"Aircraft observed a UAS off the left at 2,400 ...",['RWY 35L'],LAX,LAX,RWY 35L,[7 mile final],12.964
9,376,Aircraft observed a white UAS off the left sid...,['RWY 25L'],LAX,LAX,RWY 25L,[3 mile final],5.556


In [48]:
# Drop 'RWYLOCATION' field as it has two columns which are 'NaN' even though there is a valid RWY mentioned in the remark
rwy102_df_MF = rwy102_df_MF.drop(columns = ['RWYLOCATION'])
rwy102_df_MF = rwy102_df_MF.drop_duplicates(subset = 'REMARKS')
rwy102_df_MF

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,UAS_Distance,Distance_Kilometers
0,112,Aircraft observed a UAS at the 2 o'clock posit...,['RWY 19R'],MKE,MKE,[1.5 mile final],2.778
1,132,Aircraft observed a silver quadcopter UAS fro...,['RWY 17R'],AUS,AUS,[7 mile final],12.964
2,157,Aircraft observed a UAS SW bound at 900 feet o...,['RWY 22'],LGA,LGA,[2.5 mile final],4.630
3,162,Aircraft observed a silver UAS on 4 mile final...,['RWY 31R'],JFK,JFK,[4 mile final],7.408
4,337,Aircraft observed a UAS while descending throu...,['RWY 24'],ACK,ACK,[2 mile final],3.704
5,338,Aircraft observed a UAS while descending throu...,['RWY 24'],ACK,ACK,[2 mile final],3.704
6,359,Aircraft observed a UAS 800 feet overhead off ...,['RWY 13'],SGJ,SGJ,[1 mile final],1.852
7,364,Aircraft observed a black and silver UAS from ...,['RWY 25L'],LAX,LAX,[2 mile final],3.704
8,366,"Aircraft observed a UAS off the left at 2,400 ...",['RWY 35L'],LAX,LAX,[7 mile final],12.964
9,376,Aircraft observed a white UAS off the left sid...,['RWY 25L'],LAX,LAX,[3 mile final],5.556


#### Merge rwy102_df_MF and airports_runways_linked.csv (air_rwy)

`Merge the two datasets on the 'IDENT' field.`

In [49]:
rwy102_airwy = pd.merge(rwy102_df_MF, air_rwy, on = 'IDENT', how = 'left').reset_index(drop=True)
rwy102_airwy.rename(columns = {'properties.DESIGNATOR': 'DESIGNATOR'}, inplace = True)
rwy102_airwy

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,UAS_Distance,Distance_Kilometers,properties.GLOBAL_ID,properties.AIRPORT_ID,DESIGNATOR,Runway.Latitude,Runway.Longitude,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,112,Aircraft observed a UAS at the 2 o'clock posit...,['RWY 19R'],MKE,MKE,[1.5 mile final],2.778,5DFFD5A2-9325-4D97-8BC6-05BC6E6D5B9C,1050EB1A-2F00-4552-822E-22DAFB9B7F76,01L/19R,-87.895715,42.941378,General Mitchell Intl,42.946932,-87.897064,KMKE
1,112,Aircraft observed a UAS at the 2 o'clock posit...,['RWY 19R'],MKE,MKE,[1.5 mile final],2.778,27AC5872-A65C-42F5-87D2-43CFBEDABC61,1050EB1A-2F00-4552-822E-22DAFB9B7F76,01R/19L,-87.891533,42.943936,General Mitchell Intl,42.946932,-87.897064,KMKE
2,112,Aircraft observed a UAS at the 2 o'clock posit...,['RWY 19R'],MKE,MKE,[1.5 mile final],2.778,34066E75-7060-4AF5-84DA-CB7AF7E360EA,1050EB1A-2F00-4552-822E-22DAFB9B7F76,07L/25R,-87.898501,42.954386,General Mitchell Intl,42.946932,-87.897064,KMKE
3,112,Aircraft observed a UAS at the 2 o'clock posit...,['RWY 19R'],MKE,MKE,[1.5 mile final],2.778,55AA4DCC-9B86-4B5E-AF38-FFC49EA1213A,1050EB1A-2F00-4552-822E-22DAFB9B7F76,07R/25L,-87.905974,42.941910,General Mitchell Intl,42.946932,-87.897064,KMKE
4,112,Aircraft observed a UAS at the 2 o'clock posit...,['RWY 19R'],MKE,MKE,[1.5 mile final],2.778,1CD25585-4D46-4140-8BDA-5586834973A0,1050EB1A-2F00-4552-822E-22DAFB9B7F76,13/31,-87.897335,42.954024,General Mitchell Intl,42.946932,-87.897064,KMKE
5,132,Aircraft observed a silver quadcopter UAS fro...,['RWY 17R'],AUS,AUS,[7 mile final],12.964,5DC72573-992B-4B49-9E21-EA53A7808382,51009197-7EEE-42C3-AB2E-497FC6EAE28C,H1,-97.660994,30.185464,Austin-Bergstrom Intl,30.194527,-97.669876,KAUS
6,132,Aircraft observed a silver quadcopter UAS fro...,['RWY 17R'],AUS,AUS,[7 mile final],12.964,56D6D4A2-5C17-4D79-8885-867C1BA09C5C,51009197-7EEE-42C3-AB2E-497FC6EAE28C,H2,-97.661055,30.187661,Austin-Bergstrom Intl,30.194527,-97.669876,KAUS
7,132,Aircraft observed a silver quadcopter UAS fro...,['RWY 17R'],AUS,AUS,[7 mile final],12.964,A1B3C406-629E-4E4A-B8C6-1D62A0AAD3BF,51009197-7EEE-42C3-AB2E-497FC6EAE28C,H3,-97.673200,30.179478,Austin-Bergstrom Intl,30.194527,-97.669876,KAUS
8,132,Aircraft observed a silver quadcopter UAS fro...,['RWY 17R'],AUS,AUS,[7 mile final],12.964,164338FB-E86A-46EE-8A03-808657A6F57F,51009197-7EEE-42C3-AB2E-497FC6EAE28C,18L/36R,-97.657557,30.188991,Austin-Bergstrom Intl,30.194527,-97.669876,KAUS
9,132,Aircraft observed a silver quadcopter UAS fro...,['RWY 17R'],AUS,AUS,[7 mile final],12.964,E6EAA572-0F82-4698-BC99-410344B37E73,51009197-7EEE-42C3-AB2E-497FC6EAE28C,18R/36L,-97.678886,30.193417,Austin-Bergstrom Intl,30.194527,-97.669876,KAUS


#### Extract Runway Designator from UAS_Location_Runways field

`We want to be able to obtain the correct latitude/longitude coordinates for the appropriate runway mentioned in the remark.`

In [50]:
rwydesignator = []

for i in range(len(rwy102_airwy)):
    rwy_split = rwy102_airwy['UAS_Location_Runways'][i].split(' ')
    rwydesignator.append(rwy_split[1][:-2])

# Add split column with RWY designators to dataframe as new field
rwy102_airwy['RWYDESIGNATOR'] = rwydesignator

In [51]:
rwy102_airwy

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,UAS_Distance,Distance_Kilometers,properties.GLOBAL_ID,properties.AIRPORT_ID,DESIGNATOR,Runway.Latitude,Runway.Longitude,NAME,LATITUDE,LONGITUDE,ICAO_ID,RWYDESIGNATOR
0,112,Aircraft observed a UAS at the 2 o'clock posit...,['RWY 19R'],MKE,MKE,[1.5 mile final],2.778,5DFFD5A2-9325-4D97-8BC6-05BC6E6D5B9C,1050EB1A-2F00-4552-822E-22DAFB9B7F76,01L/19R,-87.895715,42.941378,General Mitchell Intl,42.946932,-87.897064,KMKE,19R
1,112,Aircraft observed a UAS at the 2 o'clock posit...,['RWY 19R'],MKE,MKE,[1.5 mile final],2.778,27AC5872-A65C-42F5-87D2-43CFBEDABC61,1050EB1A-2F00-4552-822E-22DAFB9B7F76,01R/19L,-87.891533,42.943936,General Mitchell Intl,42.946932,-87.897064,KMKE,19R
2,112,Aircraft observed a UAS at the 2 o'clock posit...,['RWY 19R'],MKE,MKE,[1.5 mile final],2.778,34066E75-7060-4AF5-84DA-CB7AF7E360EA,1050EB1A-2F00-4552-822E-22DAFB9B7F76,07L/25R,-87.898501,42.954386,General Mitchell Intl,42.946932,-87.897064,KMKE,19R
3,112,Aircraft observed a UAS at the 2 o'clock posit...,['RWY 19R'],MKE,MKE,[1.5 mile final],2.778,55AA4DCC-9B86-4B5E-AF38-FFC49EA1213A,1050EB1A-2F00-4552-822E-22DAFB9B7F76,07R/25L,-87.905974,42.941910,General Mitchell Intl,42.946932,-87.897064,KMKE,19R
4,112,Aircraft observed a UAS at the 2 o'clock posit...,['RWY 19R'],MKE,MKE,[1.5 mile final],2.778,1CD25585-4D46-4140-8BDA-5586834973A0,1050EB1A-2F00-4552-822E-22DAFB9B7F76,13/31,-87.897335,42.954024,General Mitchell Intl,42.946932,-87.897064,KMKE,19R
5,132,Aircraft observed a silver quadcopter UAS fro...,['RWY 17R'],AUS,AUS,[7 mile final],12.964,5DC72573-992B-4B49-9E21-EA53A7808382,51009197-7EEE-42C3-AB2E-497FC6EAE28C,H1,-97.660994,30.185464,Austin-Bergstrom Intl,30.194527,-97.669876,KAUS,17R
6,132,Aircraft observed a silver quadcopter UAS fro...,['RWY 17R'],AUS,AUS,[7 mile final],12.964,56D6D4A2-5C17-4D79-8885-867C1BA09C5C,51009197-7EEE-42C3-AB2E-497FC6EAE28C,H2,-97.661055,30.187661,Austin-Bergstrom Intl,30.194527,-97.669876,KAUS,17R
7,132,Aircraft observed a silver quadcopter UAS fro...,['RWY 17R'],AUS,AUS,[7 mile final],12.964,A1B3C406-629E-4E4A-B8C6-1D62A0AAD3BF,51009197-7EEE-42C3-AB2E-497FC6EAE28C,H3,-97.673200,30.179478,Austin-Bergstrom Intl,30.194527,-97.669876,KAUS,17R
8,132,Aircraft observed a silver quadcopter UAS fro...,['RWY 17R'],AUS,AUS,[7 mile final],12.964,164338FB-E86A-46EE-8A03-808657A6F57F,51009197-7EEE-42C3-AB2E-497FC6EAE28C,18L/36R,-97.657557,30.188991,Austin-Bergstrom Intl,30.194527,-97.669876,KAUS,17R
9,132,Aircraft observed a silver quadcopter UAS fro...,['RWY 17R'],AUS,AUS,[7 mile final],12.964,E6EAA572-0F82-4698-BC99-410344B37E73,51009197-7EEE-42C3-AB2E-497FC6EAE28C,18R/36L,-97.678886,30.193417,Austin-Bergstrom Intl,30.194527,-97.669876,KAUS,17R


#### Check if RWY designator found in REMARK matches one of the RWY designators from the rwy.csv dataset

`If YES: RWYinREMARK field is 'True'`

`If NO: RWYinREMARK field is 'False'`
***
***NOTE:*** Since we linked on the 'IDENT' field, there are repeated remarks as a result of there being multiple runways at
each airport. Thus, we are only interested in the records for which the 'RWYinREMARK' field is 'True'.

In [52]:
rwy102_airwy['RWYinREMARK'] = rwy102_airwy.apply(lambda x: str(x.RWYDESIGNATOR) in str(x.DESIGNATOR), axis=1)

# Only keep records for which the RWY Designator is present in the DESIGNATOR field 
rwy10 = rwy102_airwy[rwy102_airwy['RWYinREMARK'] == True].drop_duplicates(subset = ['REMARKS']).reset_index(drop=True)

In [53]:
rwy10

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,UAS_Distance,Distance_Kilometers,properties.GLOBAL_ID,properties.AIRPORT_ID,DESIGNATOR,Runway.Latitude,Runway.Longitude,NAME,LATITUDE,LONGITUDE,ICAO_ID,RWYDESIGNATOR,RWYinREMARK
0,112,Aircraft observed a UAS at the 2 o'clock posit...,['RWY 19R'],MKE,MKE,[1.5 mile final],2.778,5DFFD5A2-9325-4D97-8BC6-05BC6E6D5B9C,1050EB1A-2F00-4552-822E-22DAFB9B7F76,01L/19R,-87.895715,42.941378,General Mitchell Intl,42.946932,-87.897064,KMKE,19R,True
1,157,Aircraft observed a UAS SW bound at 900 feet o...,['RWY 22'],LGA,LGA,[2.5 mile final],4.630,E79935ED-5475-48DD-85FD-06D6D84D6AB4,E866CAA3-781C-4162-8C31-D16CC9E3E92E,04/22,-73.878696,40.775660,Laguardia,40.777250,-73.872611,KLGA,22,True
2,162,Aircraft observed a silver UAS on 4 mile final...,['RWY 31R'],JFK,JFK,[4 mile final],7.408,A2421FD0-7187-4324-A544-14659D6BCCBE,DD5FB154-0787-4B0F-8554-C033FAFA0155,13L/31R,-73.777891,40.652110,John F Kennedy Intl,40.639928,-73.778693,KJFK,31R,True
3,337,Aircraft observed a UAS while descending throu...,['RWY 24'],ACK,ACK,[2 mile final],3.704,C7EC798C-CBD0-4528-99CA-252048E8B927,07ABFEB8-85A2-4FCB-BA5F-583803FE3FEB,06/24,-70.066003,41.251812,Nantucket Meml,41.252975,-70.059911,KACK,24,True
4,338,Aircraft observed a UAS while descending throu...,['RWY 24'],ACK,ACK,[2 mile final],3.704,C7EC798C-CBD0-4528-99CA-252048E8B927,07ABFEB8-85A2-4FCB-BA5F-583803FE3FEB,06/24,-70.066003,41.251812,Nantucket Meml,41.252975,-70.059911,KACK,24,True
5,359,Aircraft observed a UAS 800 feet overhead off ...,['RWY 13'],SGJ,SGJ,[1 mile final],1.852,F5F77FD6-9379-4988-B64B-05095A2BCCB0,556BB8E8-4A7D-4846-A3A2-632E4F29C832,13/31,-81.338523,29.959573,Northeast Florida Rgnl,29.959252,-81.339732,KSGJ,13,True
6,364,Aircraft observed a black and silver UAS from ...,['RWY 25L'],LAX,LAX,[2 mile final],3.704,8ED22315-4327-432F-9C74-65E1A06D97A5,2C5866F1-5E27-4032-9AF8-988CE5DB82BA,07R/25L,-118.404501,33.935085,Los Angeles Intl,33.942496,-118.408049,KLAX,25L,True
7,376,Aircraft observed a white UAS off the left sid...,['RWY 25L'],LAX,LAX,[3 mile final],5.556,8ED22315-4327-432F-9C74-65E1A06D97A5,2C5866F1-5E27-4032-9AF8-988CE5DB82BA,07R/25L,-118.404501,33.935085,Los Angeles Intl,33.942496,-118.408049,KLAX,25L,True
8,392,Aircraft observed a UAS from the 12 o'clock wh...,['RWY 22R'],EWR,EWR,[3 mile final],5.556,EA4D9B89-9E47-4056-B67F-4B17812C60DB,6AB3CFCC-F6EE-4B71-A201-2DCA9BB7F2F1,04L/22R,-74.172492,40.686242,Newark Liberty Intl,40.692480,-74.168687,KEWR,22R,True
9,563,Aircraft reported a UAS at 1500 ft. while on 3...,['RWY 25R'],ENW,ENW,[3 mile final],5.556,AE932899-2A43-42D5-8AD6-DDFBF1EBF662,9F9B4071-CDD9-4647-8EFF-71E8432B7D35,07L/25R,-87.932913,42.596187,Kenosha Rgnl,42.596063,-87.927324,KENW,25R,True


#### Merge air_dec with rwy10

`Merge the two datasets on the 'IDENT' field.`

`air_dec --> Contains the cleaned airports supplemental dataset with the Declination value at the airport`

`rwy10 --> Dataframe containing 10 records referencing Runways that contain all relevant information (from the 102 records containing NO UAS_Distance information)`

In [54]:
rwy10_dec = pd.merge(rwy10, air_dec, on = 'IDENT', how = 'left')

# Only keep necessary columns/fields of interest
rwy10_decDF = pd.DataFrame(rwy10_dec[['REMARKS', 'UASLOCATION', 'IDENT', 'UAS_Location_Runways', 'UAS_Distance', 'Distance_Kilometers', 'RWYDESIGNATOR', 'Runway.Latitude', 'Runway.Longitude', 'DECLINATION']])

### Calculate Bearing Information for Runways

`To do this, you take the number portion of the Runway Designator (in the 'RWYDESIGNATOR' field) and multiply it by 10 (effectively adding a 0 to the end) and add the DECLINATION to this number`

***Example:*** *RWY 32R has designator 32R which indicates a bearing of 320 degrees*

In [55]:
designator = rwy10_decDF['RWYDESIGNATOR']
designator_bearing = []

for i in range(len(rwy10_decDF)):
    designator_bearing.append(int(re.sub("\D","",designator[i]))*10 + rwy10_decDF['DECLINATION'][i])

rwy10_decDF['RWY_BEARING'] = designator_bearing

In [56]:
rwy10_decDF.rename(columns = {'Runway.Latitude': 'Runway_Longitude', 'Runway.Longitude' : 'Runway_Latitude'}, inplace = True)

### Calculate UAS Latitude/Longitude Information using geopy library

In [57]:
uas_lat = []
uas_long = []


for i in range(len(rwy10_decDF)):
    lat_rwy = pd.to_numeric(rwy10_decDF['Runway_Latitude'][i])
    long_rwy = pd.to_numeric(rwy10_decDF['Runway_Longitude'][i])
    b = pd.to_numeric(rwy10_decDF['RWY_BEARING'][i])
    d = pd.to_numeric(rwy10_decDF['Distance_Kilometers'][i])

    origin = geopy.Point(lat_rwy, long_rwy)
    destination = geodesic(kilometers=d).destination(origin,b)

    lat2, lon2, = destination.latitude, destination.longitude

    uas_lat.append(lat2)
    uas_long.append(lon2)

# Append UAS Lat/Long information to DataFrame
rwy10_decDF['UAS_Latitude'] = uas_lat
rwy10_decDF['UAS_Longitude'] = uas_long

In [58]:
rwy10_decDF.drop_duplicates(subset = ['REMARKS']).reset_index(drop=True)

,REMARKS,UASLOCATION,IDENT,UAS_Location_Runways,UAS_Distance,Distance_Kilometers,RWYDESIGNATOR,Runway_Longitude,Runway_Latitude,DECLINATION,RWY_BEARING,UAS_Latitude,UAS_Longitude
0,Aircraft observed a UAS at the 2 o'clock posit...,MKE,MKE,['RWY 19R'],[1.5 mile final],2.778,19R,-87.895715,42.941378,-3.97494,186.02506,42.916510,-87.899286
1,Aircraft observed a UAS SW bound at 900 feet o...,LGA,LGA,['RWY 22'],[2.5 mile final],4.630,22,-73.878696,40.775660,-12.75395,207.24605,40.738590,-73.903791
2,Aircraft observed a silver UAS on 4 mile final...,JFK,JFK,['RWY 31R'],[4 mile final],7.408,31R,-73.777891,40.652110,-12.77039,297.22961,40.682607,-73.855809
3,Aircraft observed a UAS while descending throu...,ACK,ACK,['RWY 24'],[2 mile final],3.704,24,-70.066003,41.251812,-14.42364,225.57636,41.228463,-70.097554
4,Aircraft observed a UAS while descending throu...,ACK,ACK,['RWY 24'],[2 mile final],3.704,24,-70.066003,41.251812,-14.42364,225.57636,41.228463,-70.097554
5,Aircraft observed a UAS 800 feet overhead off ...,SGJ,SGJ,['RWY 13'],[1 mile final],1.852,13,-81.338523,29.959573,-6.83122,123.16878,29.950432,-81.322464
6,Aircraft observed a black and silver UAS from ...,LAX,LAX,['RWY 25L'],[2 mile final],3.704,25L,-118.404501,33.935085,11.71212,261.71212,33.930265,-118.444143
7,Aircraft observed a white UAS off the left sid...,LAX,LAX,['RWY 25L'],[3 mile final],5.556,25L,-118.404501,33.935085,11.71212,261.71212,33.927851,-118.463962
8,Aircraft observed a UAS from the 12 o'clock wh...,EWR,EWR,['RWY 22R'],[3 mile final],5.556,22R,-74.172492,40.686242,-12.59521,207.40479,40.641820,-74.202724
9,Aircraft reported a UAS at 1500 ft. while on 3...,ENW,ENW,['RWY 25R'],[3 mile final],5.556,25R,-87.932913,42.596187,-3.90298,246.09702,42.575905,-87.994783


In [59]:
# Export to .csv 
rwy10_decDF.to_csv('rwy10_uasLatLong.csv', index = False)

#### Obtain Dataframe (and corresponding .csv file) for the 2 records for which UAS Lat/Long information was not captured

In [60]:
rwy_unid2 = rwy102_df_MF.iloc[[1, 9]]

In [61]:
rwy_unid2

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,UAS_Distance,Distance_Kilometers
1,132,Aircraft observed a silver quadcopter UAS fro...,['RWY 17R'],AUS,AUS,[7 mile final],12.964
9,376,Aircraft observed a white UAS off the left sid...,['RWY 25L'],LAX,LAX,[3 mile final],5.556


In [62]:
# Export to .csv 
rwy_unid2.to_csv('rwy2_EXCEPTIONS.csv', index = False)

## Exploring the 90 records for which there is truly no Distance information in the Remark

`These 90 records are a subset of the 102 records initially thought to contain no Distance information regarding the UAS sighting`

In [63]:
rwy_unid90 = rwy102_df[rwy102_df['UAS_Distance'].isnull()].reset_index(drop=True)

In [64]:
rwy_unid90

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance
0,97,Aircraft observed a flight of 2 white UAS on t...,['RWY 24'],CRQ,CRQ,RWY 24,NaN
1,104,Aircraft observed a UAS off the left side whil...,['RWY 23L'],RDU,RDU,RWY 23L,NaN
2,107,Aircraft observed a UAS while NE bound at 900 ...,['RWY 28'],MYF,MYF,RWY 28,NaN
3,111,"Aircraft observed a UAS while S bound at 1,000...",['RWY 26R'],LGB,LGB,RWY 26R,NaN
4,114,Aircraft observed a UAS over the yop while NE ...,['RWY 22'],LWB,LWB,RWY 22,NaN
...,...,...,...,...,...,...,...
85,589,Aircraft reported a red and white quad-copter ...,['RWY 15'],PNE,PNE,RWY 15,NaN
86,590,"Aircraft reported a UAS off left side, while o...",['RWY 35L'],DWH,DWH,RWY 35L,NaN
87,593,"Aircraft reported a UAS at 4,400 feet while on...",['RWY 35C'],DFW,DFW,RWY 35C,NaN
88,597,Aircraft reported a grey quad-copter UAS off t...,['RWY 27L'],SFB,SFB,RWY 27L,NaN


In [65]:
# Export to .csv 
rwy_unid90.to_csv('rwy90_unid.csv', index = False)

# Look at the records for which there is NOT a valid IDENT

`There are 302 records (within the 504 unidentified runway remarks) that have valid IDENT/REPORTINGFACILITY Information associated with the Runway mentioned within the Remark.`

In [66]:
rwy302 = rwy_unid504[rwy_unid504['IDENT'].isnull()].reset_index(drop=True)
rwy302

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance
0,95,Aircraft observed a white UAS at 500 feet whil...,['RWY 9L'],NaN,NaN,NaN,[]
1,96,Aircraft observed a UAS off the right side whi...,['RWY 24R'],NaN,NaN,NaN,['5NM']
2,98,Aircraft observed a black Quad-copter UAS off ...,['RWY17L'],NaN,NaN,NaN,[]
3,99,Aircraft observed a white UAS off the left sid...,['RWY 8L'],NaN,NaN,NaN,['3NM']
4,100,Aircraft observed a UAS 100 feet below while o...,['RWY29'],NaN,NaN,NaN,[]
...,...,...,...,...,...,...,...
297,587,Aircraft reported a green quad-copter UAS off ...,['RWY 4L'],NaN,NaN,NaN,['2 NM']
298,592,Aircraft reported a white quad-copter UAS off ...,['RWY 10'],NaN,NaN,NaN,['1NM']
299,595,Aircraft reported a black quad-copter UAS appr...,['RWY 7'],NaN,NaN,NaN,['2 NM']
300,598,Aircraft reported a black quad-copter UAS off ...,['RWY 3R'],NaN,NaN,NaN,[]
